# Sentiment Classification


## Loading the dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.datasets import imdb
import numpy as np
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

vocab_size = 10000 #vocab size

Using TensorFlow backend.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [0]:
len(x_train[0])

218

## Train test split

In [0]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [0]:
print(x_train.shape)
print(x_test.shape)

(25000, 300)
(25000, 300)


In [0]:
# Get the words and create a dictionary with index as key and the word as value
words=imdb.get_word_index()
print(words['the'])

index=dict([(v,k) for (k,v) in words.items() if v<=10000])
index[1]

1654784/1641221 [==============================] - 1s 1us/step
1


u'the'

In [0]:
len(index)

10000

In [0]:
# Load the emneddings from Glove
from zipfile import ZipFile

with ZipFile('/content/drive/My Drive/ML_Project/NLP/NLP-1 Sentiment Classification/glove.6B.zip') as f:
  f.extractall()

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')

for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 400000 word vectors.


In [0]:
# Map the embedding vector with the words in the index:word dictionary defined before and store in embedding_matrix
embedding_matrix = np.zeros((vocab_size+1, 100))

for i,word in index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [0]:
print(embedding_matrix.shape)

(10001, 100)


## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed,Bidirectional
from tensorflow.keras.layers import Embedding
import tensorflow as tf

tf.reset_default_graph()

input_layer=Input(shape=(maxlen,),dtype=tensorflow.int64)

In [0]:
# Embedding layer 1 : With the embeddings from Glove
# Embedding layer 2 : Without the glove embeddings

#embedding = Embedding(vocab_size+1,100,weights=[embedding_matrix],trainable=True, input_length=maxlen)(input_layer)
embedding = Embedding(vocab_size+1,100,trainable=True, input_length=maxlen)(input_layer)

W0315 06:43:24.685200 140141820991360 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/keras/initializers.py:119: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0315 06:43:24.702342 140141820991360 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling __init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
lstm_Units = Bidirectional(LSTM(20))(embedding)

W0315 06:43:30.092116 140141820991360 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0315 06:43:30.097071 140141820991360 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0315 06:43:30.101047 140141820991360 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and wil

In [0]:
# Output with 2 probablities
out = (Dense(2, activation="softmax"))(lstm_Units)

In [0]:
from tensorflow.keras import backend as K
def custom_sparse_categorical_accuracy(y_true, y_pred):
    return K.cast(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())),
                  K.floatx())

In [0]:
model = Model(input_layer, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[custom_sparse_categorical_accuracy])

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 100)          1000100   
_________________________________________________________________
bidirectional (Bidirectional (None, 40)                19360     
_________________________________________________________________
dense (Dense)                (None, 2)                 82        
Total params: 1,019,542
Trainable params: 1,019,542
Non-trainable params: 0
_________________________________________________________________


In [0]:
x_train=x_train[:10000,:]
y_train=y_train[:10000]

In [0]:
import numpy as np
model.fit(np.array(x_train),
          np.array(y_train),batch_size=20, epochs=3, verbose=1)

W0315 06:43:49.808435 140141820991360 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 554s 55ms/sample - loss: 0.4585 - custom_sparse_categorical_accuracy: 0.7751
Epoch 2/3
10000/10000 [==============================] - 530s 53ms/sample - loss: 0.2477 - custom_sparse_categorical_accuracy: 0.9050
Epoch 3/3
10000/10000 [==============================] - 527s 53ms/sample - loss: 0.1735 - custom_sparse_categorical_accuracy: 0.9350


In [0]:
x_test=x_test[:5000,:]
y_test=y_test[:5000]

In [0]:
test_pred = model.predict(np.array(x_test), verbose=1)

5000/5000 [==============================] - 33s 7ms/sample


In [0]:
# Mapping the greater value as 1 and the lesser probability as 0
test_pred = [1 if j>i else 0 for i,j in test_pred]

In [0]:
test_pred[1]

1

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, test_pred)

array([[2189,  382],
       [ 383, 2046]])

## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [0]:
model.layers[2].output

<tf.Tensor 'bidirectional/concat:0' shape=(?, 40) dtype=float32>

In [0]:
layer_outputs=[out.output for out in model.layers][1:]
model_input = model.input

In [0]:
from tensorflow.keras import backend as K

functions = [K.function([model_input],[out]) for out in layer_outputs]

In [0]:
outs = [func([x_test]) for func in functions]

In [0]:
# Output of Bidirectional layer
print(outs[1])

[array([[-0.07896686, -0.02255411, -0.16243619, ...,  0.5637904 ,
        -0.36678502,  0.540138  ],
       [ 0.46420056,  0.27471346,  0.2940872 , ...,  0.5640819 ,
        -0.36701006,  0.54027355],
       [ 0.19531415,  0.0924695 ,  0.15552881, ...,  0.6093784 ,
        -0.4888146 ,  0.5793847 ],
       ...,
       [ 0.54487234,  0.32104543,  0.2967749 , ...,  0.5637904 ,
        -0.36678502,  0.540138  ],
       [ 0.32846898,  0.21101789,  0.2635395 , ...,  0.56379056,
        -0.36678502,  0.5401396 ],
       [ 0.5080348 ,  0.33299533,  0.31679457, ...,  0.5637913 ,
        -0.36678526,  0.5401465 ]], dtype=float32)]


In [0]:
# Output of Dense softmax layer
print(outs[2])

[array([[0.73686045, 0.26313952],
       [0.00731144, 0.9926886 ],
       [0.11612445, 0.88387555],
       ...,
       [0.0035222 , 0.9964778 ],
       [0.03168906, 0.9683109 ],
       [0.00385282, 0.99614716]], dtype=float32)]


In [0]:
# Output of the embedding layer
print(outs[0])

[array([[[ 0.01533807,  0.01742511, -0.06591542, ...,  0.05057117,
          0.07147822, -0.02357107],
        [ 0.01533807,  0.01742511, -0.06591542, ...,  0.05057117,
          0.07147822, -0.02357107],
        [ 0.01533807,  0.01742511, -0.06591542, ...,  0.05057117,
          0.07147822, -0.02357107],
        ...,
        [ 0.06771383,  0.03967149, -0.02217273, ...,  0.0408648 ,
          0.00991517, -0.05539263],
        [-0.02427202, -0.00758891,  0.02094026, ..., -0.04588254,
          0.01226688, -0.01142114],
        [ 0.04047591,  0.04239246,  0.1438005 , ..., -0.06521432,
          0.00739498, -0.02556022]],

       [[ 0.01533807,  0.01742511, -0.06591542, ...,  0.05057117,
          0.07147822, -0.02357107],
        [ 0.01533807,  0.01742511, -0.06591542, ...,  0.05057117,
          0.07147822, -0.02357107],
        [ 0.01533807,  0.01742511, -0.06591542, ...,  0.05057117,
          0.07147822, -0.02357107],
        ...,
        [ 0.04809335,  0.04811456,  0.00341128, ..., 